In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils
import algo
import pickle
import pandas as pd
import os
%matplotlib widget

In [ ]:
def further_regress(eeg_multisubj, eog_multisubj, L_EEG=3, L_EOG=3):
    _, _, N = eeg_multisubj.shape
    eeg_clean_list = []
    for n in range(N):
        eeg = utils.block_Hankel(eeg_multisubj[:,:,n], L_EEG)
        eog = utils.block_Hankel(eog_multisubj[:,:,n], L_EOG)
        eeg_clean = utils.regress_out(eeg, eog)
        eeg_clean_list.append(eeg_clean)
    eeg_clean_multisubj = np.stack(eeg_clean_list, axis=2)
    return eeg_clean_multisubj

## Load data

In [ ]:
subjects = ['Pilot_1', 'Pilot_2', 'Pilot_4', 'Pilot_5', 'Pilot_6', 'Pilot_7']
PATTERN = 'Overlay'
SINGLEOBJ = True
subj_path = ['../../Experiments/data/Two_Obj/' + PATTERN + '/' + sub + '/' for sub in subjects]
nb_subj = len(subjects)
bads = [['A30', 'B25'], ['B25'], ['B25'], [], ['A31', 'B31'], ['B25']]
fsStim = 30
feats_path_folder = '../Feat_Multi/features/'

In [ ]:
%%capture
eeg_multisubj_list, eog_multisubj_list, feat_all_att_list, feat_all_unatt_list, gaze_multisubj_list, fs, len_seg_list = utils.load_data(subj_path, fsStim, bads, feats_path_folder, PATTERN, singleobj=False, LOAD_ONLY=True, ALL_NEW=False)
eeg_multisubj_SO_list, eog_multisubj_SO_list, feat_all_SO_list, _, gaze_multisubj_SO_list, fs, len_seg_SO_list = utils.load_data(subj_path, fsStim, bads, feats_path_folder, PATTERN, singleobj=True, LOAD_ONLY=True, ALL_NEW=False)

In [ ]:
# eeg_clean_list = [further_regress(eeg_multisubj, eog_multisubj) for eeg_multisubj, eog_multisubj in zip(eeg_multisubj_list, eog_multisubj_list)]

In [ ]:
gaze_velocity_list = [utils.calcu_gaze_velocity(gaze) for gaze in gaze_multisubj_list]
saccade_multisubj_list = [np.expand_dims(gaze[:,2,:], axis=1) for gaze in gaze_multisubj_list]
blink_multisubj_list = [np.expand_dims(gaze[:,3,:], axis=1) for gaze in gaze_multisubj_list]
saccade_multisubj_list = utils.refine_saccades(saccade_multisubj_list, blink_multisubj_list)
eog_velocity_list = [utils.calcu_gaze_vel_from_EOG(eog) for eog in eog_multisubj_list]
gaze_velocity_list = [utils.interpolate_blinks(gaze_velocity, blink) for gaze_velocity, blink in zip(gaze_velocity_list, blink_multisubj_list)]
eog_velocity_list = [utils.interpolate_blinks(eog_velocity, blink) for eog_velocity, blink in zip(eog_velocity_list, blink_multisubj_list)] # Exist misalignment between eog and gaze
data_list = [eeg_multisubj_list, eog_multisubj_list, gaze_velocity_list, eog_velocity_list, saccade_multisubj_list, feat_all_att_list, feat_all_unatt_list]

In [ ]:
if SINGLEOBJ:
    gaze_velocity_list_SO = [utils.calcu_gaze_velocity(gaze) for gaze in gaze_multisubj_SO_list]
    saccade_multisubj_list_SO = [np.expand_dims(gaze[:,2,:], axis=1) for gaze in gaze_multisubj_SO_list]
    blink_multisubj_list_SO = [np.expand_dims(gaze[:,3,:], axis=1) for gaze in gaze_multisubj_SO_list]
    saccade_multisubj_list_SO = utils.refine_saccades(saccade_multisubj_list_SO, blink_multisubj_list_SO)
    eog_velocity_list_SO = [utils.calcu_gaze_vel_from_EOG(eog) for eog in eog_multisubj_SO_list]
    gaze_velocity_list_SO = [utils.interpolate_blinks(gaze_velocity, blink) for gaze_velocity, blink in zip(gaze_velocity_list_SO, blink_multisubj_list_SO)]
    eog_velocity_list_SO = [utils.interpolate_blinks(eog_velocity, blink) for eog_velocity, blink in zip(eog_velocity_list_SO, blink_multisubj_list_SO)] # Exist misalignment between eog and gaze
    data_list = [eeg_multisubj_list, eog_multisubj_list, gaze_velocity_list, eog_velocity_list, saccade_multisubj_list, feat_all_att_list, feat_all_unatt_list, eeg_multisubj_SO_list, eog_multisubj_SO_list, gaze_velocity_list_SO, eog_velocity_list_SO, saccade_multisubj_list_SO, feat_all_SO_list]

In [ ]:
subj_to_check = 'Pilot_5'
video_id = 3
subj_ID = subjects.index(subj_to_check)
gaze_subj = [gaze[:,:,subj_ID] for gaze in gaze_multisubj_list]
gaze_velocity_subj = [gaze_velocity[:,:,subj_ID] for gaze_velocity in gaze_velocity_list]
saccade_subj = [saccade[...,subj_ID] for saccade in saccade_multisubj_list]
eog_subj = [eog[...,subj_ID] for eog in eog_multisubj_list]
plt.close()
plt.plot(gaze_subj[video_id][:,1], label='gaze_y')
# plt.plot(gaze_velocity_subj[video_id], label='gaze_velocity')
plt.plot(saccade_subj[video_id]*1000, label='saccade')
# plt.plot((eog_subj[video_id][:,0]-eog_subj[video_id][:,1])*5e6, label='EOG_y')
plt.legend()
plt.show()

In [ ]:
# Check the alignment between eog and gaze. The synchronization is good if the peaks of two signals (eye blinks) are aligned.
# subj_to_check = 'Pilot_7'
# subj_ID = subjects.index(subj_to_check)
# utils.check_alignment(subj_ID, eog_multisubj_list, gaze_multisubj_list, nb_points=1500)

In [ ]:
RemoveSC = True
if RemoveSC:
    for data in data_list:
        data = [utils.remove_shot_cuts(d, fs) for d in data]

In [ ]:
objflow_att_list = [feats[:,8] for feats in feat_all_att_list]
objtempctr_att_list = [feats[:,17] for feats in feat_all_att_list]
objflow_unatt_list = [feats[:,8] for feats in feat_all_unatt_list]
objtempctr_unatt_list = [feats[:,17] for feats in feat_all_unatt_list]
if SINGLEOBJ:
    objflow_att_SO_list = [feats[:,8] for feats in feat_all_SO_list]
    objtempctr_att_SO_list = [feats[:,17] for feats in feat_all_SO_list]

In [ ]:
L_EEG = 3 
L_Stim = int(fsStim/2) 
offset_EEG = 1 
offset_Stim = 0 
trial_len_list = list(range(5, 50, 5))

In [ ]:
# Results saving related parameters
CLEAR = False
data_types = ['EEG', 'EOG', 'GAZE_V', 'EOG_V', 'SACC']
figure_dirs = {}
table_dirs = {}
for data_type in data_types:
    figure_path = f'figures/{PATTERN}/{data_type}/'
    utils.create_dir(figure_path, CLEAR)
    figure_dirs[data_type] = figure_path
    table_path = f'tables/{PATTERN}/{data_type}/'
    utils.create_dir(table_path, CLEAR)
    table_dirs[data_type] = table_path

## Functions

In [ ]:
def pipe_att_or_unatt_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT, n_components=3, saccade_multisubj_list=None, V_eeg=None, V_Stim=None, PLOT=False, figure_dir=None, SAVERES=False, table_dir=None):
    eeg_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_multisubj_list]
    if saccade_multisubj_list is not None:
        saccade_onesubj_list = [saccade[:,:,Subj_ID] for saccade in saccade_multisubj_list]
        mask_list = utils.get_mask_list(saccade_onesubj_list, before=10, after=20)
    else:
        mask_list = None
    CCA = algo.CanonicalCorrelationAnalysis(eeg_onesubj_list, feat_att_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, n_components=n_components, mask_list=mask_list)
    corr_att_fold, corr_unatt_fold, sig_corr_fold, forward_model_fold = CCA.att_or_unatt_LVO(feat_unatt_list, TRAIN_WITH_ATT, V_eeg=V_eeg, V_Stim=V_Stim)
    train_type = 'SO' if V_eeg is not None else 'Att' if TRAIN_WITH_ATT else 'Unatt'
    ifmask = True if mask_list is not None else False
    if PLOT:
        figure_name = f"{figure_dir}Subj_{Subj_ID+1}_Train_{train_type}_Mask_{ifmask}_Folds.png"
        utils.plot_spatial_resp_fold(forward_model_fold, corr_att_fold, corr_unatt_fold, sig_corr_fold, figure_name, AVG=False)
        utils.plot_spatial_resp_fold(forward_model_fold, corr_att_fold, corr_unatt_fold, sig_corr_fold, figure_name, AVG=True)
    if SAVERES:
        table_name = table_dir + f'Corr_Train_{train_type}_Mask_{ifmask}.csv'
        # check if the file exists
        if not os.path.isfile(table_name):
            # create a pandas dataframe that contains Subj_ID, Corr_Att, Corr_Unatt, Sig_Corr
            res_df = utils.create_corr_df(Subj_ID, sig_corr_fold, corr_att_fold, corr_unatt_fold)
        else:
            # read the dataframe
            res_df = pd.read_csv(table_name, header=0, index_col=[0,1,2])
            if not 'Subj '+str(Subj_ID+1) in res_df.index.get_level_values('Subject ID'):
                res_add = utils.create_corr_df(Subj_ID, sig_corr_fold, corr_att_fold, corr_unatt_fold)
                res_df = pd.concat([res_df, res_add], axis=0)
        with open(table_name, 'w') as f:
            res_df.to_csv(f, header=True)

In [ ]:
def pipe_compete_trials_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir, saccade_multisubj_list=None, BOOTSTRAP=True, V_eeg=None, V_Stim=None, n_components=3, nb_comp_into_account=3, signifi_level=False, message=True):
    res = []
    eeg_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_multisubj_list]
    if saccade_multisubj_list is not None:
        saccade_onesubj_list = [saccade[:,:,Subj_ID] for saccade in saccade_multisubj_list]
        mask_list = utils.get_mask_list(saccade_onesubj_list, before=10, after=20)
    else:
        mask_list = None
    ifmask = True if mask_list is not None else False
    CCA = algo.CanonicalCorrelationAnalysis(eeg_onesubj_list, feat_att_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, n_components=n_components, mask_list=mask_list, signifi_level=signifi_level, message=message)
    for trial_len in trial_len_list:
        print('Trial length: ', trial_len)
        corr_att_eeg, corr_unatt_eeg = CCA.att_or_unatt_LVO_trials(feat_unatt_list, trial_len=trial_len, BOOTSTRAP=BOOTSTRAP, V_eeg=V_eeg, V_Stim=V_Stim)
        acc, _, _, _, _= utils.eval_compete(corr_att_eeg, corr_unatt_eeg, TRAIN_WITH_ATT=True, nb_comp_into_account=nb_comp_into_account)
        res.append(acc)
    train_type = 'SO' if V_eeg is not None else 'Att'
    table_name = table_dir + f'Acc_Train_{train_type}_Mask_{ifmask}.csv'
    if not os.path.isfile(table_name):
        # create a pandas dataframe that contains Subj_ID, Corr_Att, Corr_Unatt, Sig_Corr
        res_df = utils.create_acc_df(Subj_ID, trial_len_list, res)
    else:
        # read the dataframe
        res_df = pd.read_csv(table_name, header=0)
        if ('Subj ' + str(Subj_ID + 1)) not in res_df['Subject ID'].values:
            res_add = utils.create_acc_df(Subj_ID, trial_len_list, res)
            res_df = pd.concat([res_df, res_add], axis=0)
    with open(table_name, 'w') as f:
        res_df.to_csv(f, header=True, index=False)

In [ ]:
def pipe_mm_trials_LVO(Subj_ID, eeg_multisubj_list, feat_match_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dir, MATCHATT, saccade_multisubj_list=None, V_eeg=None, V_Stim=None, n_components=3, nb_comp_into_account=3, signifi_level=False, message=True):
    res = []
    eeg_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_multisubj_list]
    if saccade_multisubj_list is not None:
        saccade_onesubj_list = [saccade[:,:,Subj_ID] for saccade in saccade_multisubj_list]
        mask_list = utils.get_mask_list(saccade_onesubj_list, before=10, after=20)
    else:
        mask_list = None
    ifmask = True if mask_list is not None else False
    CCA = algo.CanonicalCorrelationAnalysis(eeg_onesubj_list, feat_match_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, n_components=n_components, mask_list=mask_list, signifi_level=signifi_level, message=message)
    for trial_len in trial_len_list:
        print('Trial length: ', trial_len)
        corr_match_eeg, corr_mismatch_eeg = CCA.match_mismatch_LVO(trial_len=trial_len, V_eeg=V_eeg, V_Stim=V_Stim)
        acc, _, _ = utils.eval_mm(corr_match_eeg, corr_mismatch_eeg, nb_comp_into_account)
        res.append(acc)
    train_type = 'SO' if V_eeg is not None else 'Att' if MATCHATT else 'Unatt'
    table_name = table_dir + f'Acc_MM_Train_{train_type}_Mask_{ifmask}.csv'
    if not os.path.isfile(table_name):
        # create a pandas dataframe that contains Subj_ID, Corr_Att, Corr_Unatt, Sig_Corr
        res_df = utils.create_acc_df(Subj_ID, trial_len_list, res)
        res_df.rename(columns={'Att': 'Match', 'Unatt': 'Mismatch'}, inplace=True)
    else:
        # read the dataframe
        res_df = pd.read_csv(table_name, header=0)
        if ('Subj ' + str(Subj_ID + 1)) not in res_df['Subject ID'].values:
            res_add = utils.create_acc_df(Subj_ID, trial_len_list, res)
            res_add.rename(columns={'Att': 'Match', 'Unatt': 'Mismatch'}, inplace=True)
            res_df = pd.concat([res_df, res_add], axis=0)
    with open(table_name, 'w') as f:
        res_df.to_csv(f, header=True, index=False)

In [ ]:
def pipe_single_obj(Subj_ID, eeg_multisubj_list, feat_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, figure_dir, table_dir, saccade_multisubj_list=None, n_components=3, PLOT=False):
    eeg_onesubj_list = [eeg[:,:,Subj_ID] for eeg in eeg_multisubj_list]
    if saccade_multisubj_list is not None:
        saccade_onesubj_list = [saccade[:,:,Subj_ID] for saccade in saccade_multisubj_list]
        mask_list = utils.get_mask_list(saccade_onesubj_list, before=10, after=20)
    else:
        mask_list = None
    ifmask = True if mask_list is not None else False
    CCA = algo.CanonicalCorrelationAnalysis(eeg_onesubj_list, feat_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, n_components=n_components, mask_list=mask_list)
    corr_train_fold, corr_test_fold, sig_corr_fold, forward_model_fold = CCA.cross_val_LVO()
    if PLOT:
        figure_name = f"{figure_dir}Subj_{Subj_ID+1}_Folds.png"
        utils.plot_spatial_resp_fold(forward_model_fold, corr_test_fold, None, sig_corr_fold, figure_name, AVG=False)
        utils.plot_spatial_resp_fold(forward_model_fold, corr_test_fold, None, sig_corr_fold, figure_name, AVG=True)
    table_name = table_dir + f'Corr_SO_Mask_{ifmask}.csv'
    # check if the file exists
    if not os.path.isfile(table_name):
        # create a pandas dataframe that contains Subj_ID, Corr_Att, Corr_Unatt, Sig_Corr
        res_df = utils.create_corr_df(Subj_ID, sig_corr_fold, corr_train_fold, corr_test_fold)
        res_df.rename(columns={'Att': 'Train', 'Unatt': 'Test'}, inplace=True)
    else:
        # read the dataframe
        res_df = pd.read_csv(table_name, header=0, index_col=[0,1,2])
        if not 'Subj '+str(Subj_ID+1) in res_df.index.get_level_values('Subject ID'):
            res_add = utils.create_corr_df(Subj_ID, sig_corr_fold, corr_train_fold, corr_test_fold)
            res_add.rename(columns={'Att': 'Train', 'Unatt': 'Test'}, inplace=True)
            res_df = pd.concat([res_df, res_add], axis=0)
    with open(table_name, 'w') as f:
        res_df.to_csv(f, header=True)
    EEG_all = np.concatenate(eeg_onesubj_list, axis=0)
    feat_all = np.concatenate(feat_list, axis=0)
    _, _, _, _, V_eeg_SO, V_stim_SO, _ = CCA.fit(EEG_all, feat_all)
    return V_eeg_SO, V_stim_SO

## If contains single object dataset

In [ ]:
if SINGLEOBJ:
    for Subj_ID in range(len(subjects)):
        feat_train_list = objflow_att_SO_list
        V_eeg_SO, V_stim_SO = pipe_single_obj(Subj_ID, eeg_multisubj_SO_list, feat_train_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, figure_dirs['EEG'], table_dirs['EEG'], saccade_multisubj_list=None, n_components=3, PLOT=True)
        feat_att_list = objflow_att_list
        feat_unatt_list = objflow_unatt_list
        pipe_att_or_unatt_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, PLOT=True, figure_dir=figure_dirs['EEG'], SAVERES=True, table_dir=table_dirs['EEG'], V_eeg=V_eeg_SO, V_Stim=V_stim_SO)
        pipe_compete_trials_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], V_eeg=V_eeg_SO, V_Stim=V_stim_SO)

## Mode=Compete: Discriminating between attended and unattended segments

In [ ]:
feat_att_list = objflow_att_list
feat_unatt_list = objflow_unatt_list

In [ ]:
for Subj_ID in range(len(subjects)):
    pipe_att_or_unatt_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, PLOT=True, figure_dir=figure_dirs['EEG'], SAVERES=True, table_dir=table_dirs['EEG'])
    pipe_att_or_unatt_LVO(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_dirs['EOG'])
    pipe_att_or_unatt_LVO(Subj_ID, gaze_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_dirs['GAZE_V'])
    pipe_att_or_unatt_LVO(Subj_ID, eog_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_dirs['EOG_V'])
    pipe_att_or_unatt_LVO(Subj_ID, saccade_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_dirs['SACC'])

In [ ]:
for Subj_ID in range(len(subjects)):
    pipe_compete_trials_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'])
    pipe_compete_trials_LVO(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EOG'])
    pipe_compete_trials_LVO(Subj_ID, gaze_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['GAZE_V'])
    pipe_compete_trials_LVO(Subj_ID, eog_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EOG_V'])
    pipe_compete_trials_LVO(Subj_ID, saccade_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['SACC'])

## Mode=Match-Mismatch: Discriminating between match and mismatch segments

In [ ]:
for Subj_ID in range(len(subjects)):
    feat_match_list = objflow_att_list
    pipe_mm_trials_LVO(Subj_ID, eeg_multisubj_list, feat_match_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], MATCHATT=True, saccade_multisubj_list=None)
    feat_match_list = objflow_unatt_list
    pipe_mm_trials_LVO(Subj_ID, eeg_multisubj_list, feat_match_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], MATCHATT=False, saccade_multisubj_list=None)

## What to do with the saccades

In [ ]:
for Subj_ID in range(len(subjects)):
    pipe_att_or_unatt_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, saccade_multisubj_list=saccade_multisubj_list, PLOT=True, figure_dir=figure_dirs['EEG'], SAVERES=True, table_dir=table_dirs['EEG'])
    pipe_att_or_unatt_LVO(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, saccade_multisubj_list=saccade_multisubj_list, SAVERES=True, table_dir=table_dirs['EOG'])
    pipe_att_or_unatt_LVO(Subj_ID, gaze_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, saccade_multisubj_list=saccade_multisubj_list, SAVERES=True, table_dir=table_dirs['GAZE_V'])
    pipe_att_or_unatt_LVO(Subj_ID, eog_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, saccade_multisubj_list=saccade_multisubj_list, SAVERES=True, table_dir=table_dirs['EOG_V'])

In [ ]:
for Subj_ID in range(len(subjects)):
    pipe_compete_trials_LVO(Subj_ID, eeg_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EEG'], saccade_multisubj_list=saccade_multisubj_list)
    pipe_compete_trials_LVO(Subj_ID, eog_multisubj_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EOG'], saccade_multisubj_list=saccade_multisubj_list)
    pipe_compete_trials_LVO(Subj_ID, gaze_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['GAZE_V'], saccade_multisubj_list=saccade_multisubj_list)
    pipe_compete_trials_LVO(Subj_ID, eog_velocity_list, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_dirs['EOG_V'], saccade_multisubj_list=saccade_multisubj_list)

## Making use of multi-modal data

In [108]:
def stack_modal(modal_nested_list):
    nb_video = len(modal_nested_list[0])
    stacked_list = []
    for i in range(nb_video):
        modal_list = [modal[i] for modal in modal_nested_list]
        modal_stacked = np.concatenate(tuple(modal_list), axis=1)
        stacked_list.append(modal_stacked)
    return stacked_list

In [109]:
modal_dict = {'EEG': eeg_multisubj_list, 'EOG': eog_multisubj_list, 'GAZE_V': gaze_velocity_list}
comb_dict = {'EEG_EOG': stack_modal([modal_dict['EEG'], modal_dict['EOG']]), 
             'EEG_GAZE_V': stack_modal([modal_dict['EEG'], modal_dict['GAZE_V']]), 
             'EOG_GAZE_V': stack_modal([modal_dict['EOG'], modal_dict['GAZE_V']]), 
             'EEG_EOG_GAZE_V': stack_modal([modal_dict['EEG'], modal_dict['EOG'], modal_dict['GAZE_V']])}

In [ ]:
for comb, modal in comb_dict.items():
    table_path = f'tables/{PATTERN}/{comb}/'
    utils.create_dir(table_path, CLEAR)
    for Subj_ID in range(len(subjects)):
        pipe_att_or_unatt_LVO(Subj_ID, modal, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, TRAIN_WITH_ATT=True, SAVERES=True, table_dir=table_path)
        pipe_compete_trials_LVO(Subj_ID, modal, feat_att_list, feat_unatt_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, trial_len_list, table_path)